In [11]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

import os

import scipy.ndimage
import xarray as xr
from collections import defaultdict
from scipy.signal import butter,filtfilt,savgol_filter, find_peaks, find_peaks_cwt, detrend, peak_prominences,peak_widths
import scipy

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display

from sklearn import linear_model

import panel as pn
import param
from pathlib import Path

import functools
import plotly.express as px
import plotly.graph_objs as go

from background_drift import subtract_background



In [0]:
pn.extension(template='fast')
pn.extension('plotly')

Define where the data currently resides - this could be done better but for now we'll give it a local directory

In [13]:
cwd=os.getcwd()
#lets do the path stuff
# browse to a directory
data_dir=Path(cwd+'/test_data/20221216')
data_files=list(data_dir.glob('*.txt'))


In [14]:
class Stage1(param.Parameterized):
    
    fig = go.Figure() 
   
    _columns=['Conductivity','Sodium','HOOH','Calcium']
 
    data_file=param.Selector(objects=list(data_dir.glob('*.txt')))
    species = param.Selector(default='Conductivity',objects=list(_columns))


    df=param.DataFrame()

    timex=param.Array()


    start_pos = param.Number()
    end_pos = param.Number()
    
       
    def sub_background(self):
        print("did something")
        self.output_array=subtract_background(self.df[self.species],self.timex,self.start_pos,self.end_pos)
        self.fig.add_trace(go.Scatter(x=self.df['Time'],y=self.output_array,mode='lines',name=self.species+"_background"))
        #trigger the view
    
    def calc_endpoints(self,data):
        window=51
        smooth=savgol_filter(data, window_length=window, polyorder=2)
        der2=savgol_filter(smooth, window_length=window, polyorder=2, deriv=2)
        abs_der2=np.abs(der2)
        max_der2 = np.max(der2)

        peaks_der2,_=find_peaks(abs_der2,height=max_der2/20,wlen=window)

        widths,heights,posl,posr=peak_widths(abs_der2,peaks_der2,rel_height=1)
        start_pos=int(np.floor(posl[0]))
        end_pos=int(np.ceil(posr[-1]))
        return start_pos,end_pos,der2

    def background_sub(self):
        window=51
        smooth=savgol_filter(data, window_length=window, polyorder=2)
        der2=savgol_filter(smooth, window_length=window, polyorder=2, deriv=2)
        abs_der2=np.abs(der2)
        max_der2 = np.max(der2)
        print("something")

    @param.depends('species','start_pos','end_pos')
    def make_plot(self):
        fig = self.fig
        fig.add_trace(go.Scatter(x=self.df['Time'],y=self.df[self.species],mode='lines',name=self.species))
        fig.add_vline(x=self.start_pos,line_color="red")
        fig.add_vline(x=self.end_pos,line_color="red")

        fig.update_layout(
            title='Test',
            xaxis_title='Time',
            showlegend=True
        )

        return pn.Column(self.param.start_pos,self.param.end_pos,pn.pane.Plotly(fig))


    @param.depends('data_file')
    def view(self):
        converters = {'MQ/Sample': str.strip, 'Time': int, 'Core Logging': str.strip,'Valve Indicator': str.strip}
        self.df=pd.read_csv(self.data_file, delimiter='\t',converters=converters)
        print(list(self.df.columns))
       # columns = list(self.df.columns)
        #self.param['species'].objects=columns
        self.species='Conductivity'
        self.timex=self.df['Time'].values
        self.param['start_pos'].bounds=(self.timex[0],self.timex[-1])
        self.param['end_pos'].bounds=(self.timex[0],self.timex[-1])

        _start_pos,_end_pos,_=self.calc_endpoints(self.df[self.species].values)
        self.start_pos=self.timex[_start_pos]
        self.end_pos=self.timex[_end_pos]
        return self.param.species

    def panel(self):
        button=pn.widgets.Button(name='Foo', button_type='success')
        button.on_click(lambda event: setattr(self,'read', True))
        pn.Column(self.param.data_file,self.view,self.make_plot,button)

In [15]:
app = pn.Row(Stage1)
app.servable()

Launching server at http://localhost:43689
